In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

## **Dataset**

In [2]:
num_inputs = 2

w = torch.tensor([2, -3.4])
b = 4.2

batch_size = 32

num_train = 1000
X_train = torch.normal(0, 1, (num_train, num_inputs))
y_train = torch.matmul(X_train, w.reshape(-1, 1)) + b + torch.normal(0, 0.01, (num_train, 1))

num_val = 1000
X_val = torch.normal(0, 1, (num_val, num_inputs))
y_val = torch.matmul(X_val, w.reshape(-1, 1)) + b + torch.normal(0, 0.01, (num_val, 1))

train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

val_ds = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

## **Functions**

In [3]:
class LinearRegression(nn.Module):
    def __init__(self,num_inputs, lr):
        super().__init__()
        self.lr = lr
        self.net = nn.Linear(num_inputs, 1)
        self.net.weight.data.normal_(0, 0.01)
        self.net.bias.data.fill_(0)

    def forward(self, X):
        return self.net(X)

## **Training**

In [4]:
model = LinearRegression(num_inputs=num_inputs, lr=0.03)
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[0.0134, 0.0019]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


In [5]:
optimizer = torch.optim.SGD(params=model.parameters(), lr=model.lr)

In [6]:
criterion = nn.MSELoss()

In [7]:
num_epochs = 10
for i in range(num_epochs):
    model.train()

    num_train_batches = 0
    train_loss = 0
    for X, y in train_loader:
        optimizer.zero_grad()
        y_hat = model(X)
        loss = criterion(y_hat, y)
        loss.backward()   # loss를 parameter로 미분해라 → w.grad, b.grad 값 생김
        optimizer.step()  # 계산한 gradient에 따라 parameter를 업데이트해라

        num_train_batches += 1
        train_loss += loss.item()  # 값만 가져와라
        
    model.eval()

    num_val_batches = 0
    val_loss = 0
    with torch.no_grad():
        for X, y in val_loader:
            y_hat = model(X)
            loss = criterion(y_hat, y)

            num_val_batches += 1
            val_loss += loss.item()

    print(f'epoch {i:02d} train_loss={train_loss/num_train_batches:.4f} val_loss={val_loss/num_val_batches:.4f}')

epoch 00 train_loss=8.7769 val_loss=0.6978
epoch 01 train_loss=0.1755 val_loss=0.0130
epoch 02 train_loss=0.0034 val_loss=0.0003
epoch 03 train_loss=0.0002 val_loss=0.0001
epoch 04 train_loss=0.0001 val_loss=0.0001
epoch 05 train_loss=0.0001 val_loss=0.0001
epoch 06 train_loss=0.0001 val_loss=0.0001
epoch 07 train_loss=0.0001 val_loss=0.0001
epoch 08 train_loss=0.0001 val_loss=0.0001
epoch 09 train_loss=0.0001 val_loss=0.0001


In [8]:
model.net.weight.data

tensor([[ 2.0002, -3.4001]])

In [9]:
w

tensor([ 2.0000, -3.4000])

In [10]:
model.net.bias.data

tensor([4.1998])

In [11]:
b

4.2